In [1]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Let's add the headers to our data for clarity
data_file_path = 'welddb/welddb.data'
headers_file_path = 'welddb/headers.txt'
with open(headers_file_path, 'r') as f:
    headers = [line.strip() for line in f]
df = pd.read_csv(data_file_path, sep=r'\s+', header=None, names=headers)

In [ ]:
df

In [ ]:
# let's see how many different sources we haves

unique = []
for i in df['Weld ID'].unique():
    if i.split('/')[0] not in unique:
        unique.append(i.split('/')[0])
unique2 = []
for i in unique:
    if i.split('-')[0] not in unique2:
        try :
            unique2.append(i.split('-')[0].concat(i.split('-')[1]))
        except:
            unique2.append(i.split('-')[0])
unique3 = []
for i in unique2:
    if i.split('+')[0] not in unique3:
        unique3.append(i.split('+')[0])
unique3

We can see that the presence of an 'N' indicates that the value was not reported in the publication. We will replace this character with a `NaN` value.


In [ ]:
len(unique3)

We have 59 different sources, this justifies why we have a lot of missing values and a lot of anomalies...

In [6]:
df.replace('N', np.nan, inplace=True)

Let's check the variable types so we can convert them to the correct format.

In [ ]:
df.dtypes.value_counts()

The output shows that 40 columns are of type `object` (likely containing strings or improperly formatted data) and 4 columns are `float64` (numeric).

We will inspect and clean the `object` columns to ensure proper numeric conversions where needed.

In [ ]:
#see the 1197th row
df.loc[1197]

Some rows that are expected to have numerical values may contain non-numerical ones (e.g the `'<0.002'` value assigned to the variable *Sulphur concentration (weight%)* in the 1197th row of the DataFrame).

First, we will convert the rows that can be converted, and then we will handle the remaining ones.


In [9]:
# Convert the variables to numeric if possible
def convert_to_numeric(column):
    try:
        return pd.to_numeric(column)
    except ValueError:
        return column  # If conversion fails, return the original column

df = df.apply(convert_to_numeric)


In [ ]:
df.dtypes.value_counts()

We conclude:

- **23 columns** are of type `float64`, indicating they have successfully been converted to numeric data types (floating-point numbers).
- **21 columns** are still of type `object`, meaning they likely contain non-numeric data, mixed types, or have values that couldn't be converted (e.g., strings or special characters).

Further cleaning may be needed for the non-numeric columns.

In [ ]:
# Select only columns of type object
object_columns = df.select_dtypes(include=['object'])

# Loop through each object column and display non-numeric values
for column in object_columns.columns:
    print(f"Non-numeric values in column '{column}':")
    non_numeric_values = df[column][pd.to_numeric(df[column], errors='coerce').isna()].unique()
    print(non_numeric_values)
    print('\n')

We can see that the non-numeric values in the `object` columns contain a mixture of:

1. **Special characters** like `<`, `+`, `-`, and ranges (e.g., `150-200`).
2. **Text-based values** (e.g., `'67tot33res'` or `'AC'`/`'DC'`).
3. **Units embedded in the values** (e.g., `'158(Hv30)'` or `'459Hv10'`).
4. **Missing values**.

1. **Special characters (e.g., `<0.002`)**:
   - First we will replace values like `<0.002` with numeric approximations (e.g., `0.002`).

In [ ]:
df.replace({r'<': '', }, regex=True, inplace=True)
df = df.apply(pd.to_numeric, errors='ignore')

In [ ]:
df.dtypes.value_counts()

2. **Ranges (e.g., `150-200`)**:
   - Now, we will extract the mean of the range or split it into two columns for the lower and upper bounds.

In fact, this variable, Interpass temperature (°C), describes the temperature of the material between multiple passes of the welding process. Maintaining a consistent interpass temperature is critical for ensuring the quality and mechanical properties of the weld. In some rows, the recorded value for this variable is given as a range, such as 150-200°C, rather than a single temperature.

Since we need a numerical value for further analysis, we will transform this interval into its median value, which in this case is 175°C. This allows us to approximate the interpass temperature while ensuring that the data remains consistent and usable for modeling purposes.

In [14]:
df1 = df.copy()

In [ ]:
# Find the non-numeric entries in 'Nitrogen concentration (ppm)'
df1['Interpass temperature (deg C)_numeric'] = pd.to_numeric(df1['Interpass temperature (deg C)'], errors='coerce')

# Identify rows where the conversion resulted in NaN (indicating non-numeric values)
problematic_entries = df1[df1['Interpass temperature (deg C)_numeric'].isna() & ~df1['Interpass temperature (deg C)'].isna()]['Interpass temperature (deg C)'].unique()

df1 = df1.drop(columns=['Interpass temperature (deg C)_numeric'])

problematic_entries  

In [ ]:
# see the rows with a value of '150-200' in 'Interpass temperature (deg C)'
df[df['Interpass temperature (deg C)'] == '150-200'].head()

In [15]:
def converter(x):
    try:
        return float(x)
    except:
        return 175

In [ ]:
df['Interpass temperature (deg C)'] = df['Interpass temperature (deg C)'].apply(converter)
df['Interpass temperature (deg C)']

In [ ]:
df.dtypes.value_counts()

3. **Text-based values (e.g., `'67tot33res'`)**:
   - We will investigate further to encode them meaningfully.

- Let's address the column: **'Nitrogen concentration (ppm)'**.

In [18]:
df1 = df.copy()

In [ ]:
# Find the non-numeric entries in 'Nitrogen concentration (ppm)'
df1['Nitrogen concentration (ppm)_numeric'] = pd.to_numeric(df1['Nitrogen concentration (ppm)'], errors='coerce')

# Identify rows where the conversion resulted in NaN (indicating non-numeric values)
problematic_entries = df1[df1['Nitrogen concentration (ppm)_numeric'].isna() & ~df1['Nitrogen concentration (ppm)'].isna()]['Nitrogen concentration (ppm)'].unique()

df1 = df1.drop(columns=['Nitrogen concentration (ppm)_numeric'])

problematic_entries  

After extensive investigation into the meaning of these values, we discovered through various articles that the notation **XtotYres** is an abbreviation for **X total** and **Y residual**. This indicates that the nitrogen concentration in the material is divided into two components: the total concentration (X) and the residual concentration (Y), which remains after some process (nd stands for non-detected).

For the purpose of our analysis, we will focus on the total concentration (X), as it represents the complete amount of nitrogen present in the material before any processes or reactions occur. The residual concentration often reflects secondary or incomplete reactions and is typically less representative of the material's initial state or overall chemical composition. By considering the total concentration, we ensure that our analysis captures the full nitrogen content, which is more relevant for evaluating the material's properties and predicting weld quality.

*(We have only 59 values containing residual concentrations, so we don't need to consider the residual concentration as a new independent variable.)*


We will therefore replace the values of these cases with the total concentration (X)

In [20]:
# convert the problematic entries to numeric
def converter(x):
    try:
        return float(x)
    except:
        return float(x[:2])

In [21]:
df['Nitrogen concentration (ppm)'] = df['Nitrogen concentration (ppm)'].apply(converter)

In [ ]:
df['Nitrogen concentration (ppm)']

Now, all **'Nitrogen concentration (ppm)'** values are numerical.


4. **Embedded units (e.g., `'158(Hv30)'`)**:
   - Now, we will investigate more on this feature.

In [23]:
df1 = df.copy()

In [ ]:
df1['Hardness (kgmm-2)_numeric'] = pd.to_numeric(df1['Hardness (kgmm-2)'], errors='coerce')

# Identify rows where the conversion resulted in NaN (indicating non-numeric values)
problematic_entries = df1[df1['Hardness (kgmm-2)_numeric'].isna() & ~df1['Hardness (kgmm-2)'].isna()]['Hardness (kgmm-2)'].unique()

df1 = df1.drop(columns=['Hardness (kgmm-2)_numeric'])

problematic_entries  

In [25]:
#Remove '(' and ')' from the 'Hardness (kgmm-2)' column
df['Hardness (kgmm-2)'] = df['Hardness (kgmm-2)'].str.replace('(', '')
df['Hardness (kgmm-2)'] = df['Hardness (kgmm-2)'].str.replace(')', '')


In [ ]:
#Plot categorical data
df['Hardness (kgmm-2)'].value_counts().plot(kind='bar')

In [ ]:
#count the number of missing values
df['Hardness (kgmm-2)'].isnull().sum()

In [28]:
df1 = df.copy()

In [ ]:
df1['Hardness (kgmm-2)_numeric'] = pd.to_numeric(df1['Hardness (kgmm-2)'], errors='coerce')

# Identify rows where the conversion resulted in NaN (indicating non-numeric values)
problematic_entries = df1[df1['Hardness (kgmm-2)_numeric'].isna() & ~df1['Hardness (kgmm-2)'].isna()]['Hardness (kgmm-2)'].unique()

df1 = df1.drop(columns=['Hardness (kgmm-2)_numeric'])

problematic_entries  


In this column, we have two types of hardness values:

1. **Numerical values**: These are already standardized, such as `257`, `153`, etc.
2. **Mixed values**: These values are formatted as `xHVy` (e.g., `150Hv30`), where `x` is the hardness value and `y` is the load in kgf used during the test.

We need to standardize all the values in this column so that they are comparable, assuming a standard load of **10 kgf**.

We will use the following formula to standardize values to a common load (e.g., 10 kgf):



$$
Hv_{\text{standard}} = Hv_{\text{measured}} \times \left( \frac{L_{\text{standard}}}{L_{\text{measured}}} \right)^n
$$

Where:
- $ Hv_{\text{standard}} $ is the standardized hardness value.
- $ Hv_{\text{measured}} $ is the hardness value measured at a specific load.
- $  L_{\text{standard}} $ is the standard load we will choose (in this case, **10 kgf**).
- $  L_{\text{measured}} $ is the load used during the measurement (extracted from the notation `xHVy`).
- $  n  $ is an empirical constant, typically around **0.2** for metals.



For values that don't mention a load (e.g., `257`), we will assume the default load used was **10 kgf**, meaning no adjustment is necessary.


In [52]:
n=0.2
L_standard = 10

def transformer(x):
    try:
        return float(x)
    except:
        if x != np.nan :
            liste=x.split('Hv')
            L_measured = float(liste[1])
            HV_measured = float(liste[0])
            return HV_measured*((L_standard/L_measured)**n)

In [ ]:
df['Hardness (kgmm-2)'] = df['Hardness (kgmm-2)'].apply(transformer)
df['Hardness (kgmm-2)'].unique()

Well done, now the hardness column is float type

In [ ]:
df.dtypes.value_counts()

5. **Non-numeric categories (e.g., `'AC'`, `'DC'`, `'+'`, `'-'`)**:
   - These categorical values can be encoded later (e.g., with `LabelEncoder` or `OneHotEncoder`).


In [ ]:
object_columns = df.select_dtypes(include=['object']).columns

df[object_columns]

In [34]:
#onehot encoding to ac vs dc and Electrode positive or negative

df222 = pd.get_dummies(df, columns=['AC or DC', 'Electrode positive or negative'])[['AC or DC_AC',	'AC or DC_DC', 'Electrode positive or negative_+', 'Electrode positive or negative_-',	'Electrode positive or negative_0']]


In [ ]:
# correlation matrix plot 

import seaborn as sns

corr = df222.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)


In [ ]:
df['AC or DC'].isna().sum()

In [ ]:
df['Electrode positive or negative'].isna().sum()

We can see that the column ac or dc has more Nan values than  the Electrode positive or negative, and since they r correlated with the eletrode column, we will consider only the electrode column

In [38]:
df.drop(columns=['AC or DC'], inplace=True)

In [39]:
df = pd.get_dummies(df, columns=['Electrode positive or negative'])

Since we want to build a prediction model, we should deal with other categorical variables, let's adress the Type of weld column

In [ ]:
# see all the unique values in the 'Type of weld' column
df['Type of weld'].unique()

In [41]:
#onehot encoding to 'Type of weld' column

df = pd.get_dummies(df, columns=['Type of weld'])

In [ ]:
# see the columns names

df.head()

In [ ]:
df.info()

Let's explore more the variables

Based on related articles in the domain of welding and materials science, it is common to handle missing values of certain chemical elements differently. For instance:

- Missing values of **Phosphorus (P)** and **Sulphur (S)** are imputed using the **mean values** from the dataset.
- Other missing values of other elements such as **Manganese (Mn)**, **Nickel (Ni)**, etc., are set to **0** because these elements are not deliberately added and are likely close to the detection limit of the analytical techniques used.

Therefore, we will implement this preprocessing step to ensure that the dataset is correctly handled for the prediction task.

In [44]:
df['Sulphur concentration (weight%)'] = df['Sulphur concentration (weight%)'].fillna(df['Sulphur concentration (weight%)'].mean())
df['Phosphorus concentration (weight%)'] = df['Phosphorus concentration (weight%)'].fillna(df['Phosphorus concentration (weight%)'].mean())

# Replace missing values for all other concentrations with 0 (as they were not deliberately added)
elements_to_zero = ['Manganese concentration (weight%)', 'Sulphur concentration (weight%)',
       'Phosphorus concentration (weight%)', 'Nickel concentration (weight%)',
       'Chromium concentration (weight%)', 'Molybdenum concentration (weight%)',
       'Vanadium concentration (weight%)', 'Copper concentration (weight%)',
       'Cobalt concentration (weight%)', 'Tungsten concentration (weight%)',
       'Oxygen concentration (ppm)', 'Titanium concentration (ppm)',
       'Nitrogen concentration (ppm)', 'Aluminium concentration (ppm)',
       'Boron concentration (ppm)', 'Niobium concentration (ppm)',
       'Tin concentration (ppm)', 'Arsenic concentration (ppm)',
       'Antimony concentration (ppm)']

# Replace missing values with 0 for the other elements
df[elements_to_zero] = df[elements_to_zero].fillna(0)

In [ ]:
df.info()

### Justification du remplacement des colonnes de tension et d'intensité par une colonne de puissance

Les colonnes de **Voltage** (en volts) et d'**Current** (en ampères) ont été remplacées par une colonne unique représentant la **Puissance** (en watts). Ce choix est justifié par la relation physique qui lie ces trois grandeurs, à savoir que la puissance électrique est définie par le produit de la tension et de l'intensité
$$
P = U \times I
$$

L'ajout d'une colonne de puissance permet de capturer l'interaction entre la tension et l'intensité dans une seule variable, ce qui réduit la redondance des données et simplifie l'analyse tout en conservant une information complète. De plus, dans de nombreux cas d'application en modélisation, la puissance est plus représentative du comportement global d'un système électrique que ses composantes individuelles (tension et intensité).


In [ ]:
df['Puissance (W)']=df['Voltage (V)']*df['Current (A)']
df['Puissance (W)']

In [47]:
df.drop(['Current (A)', 'Voltage (V)', 'Weld ID'], axis=1, inplace=True)


In [48]:
bool_columns = df.select_dtypes(include='bool').columns


In [ ]:
# Convert all boolean columns to integers (0 for False, 1 for True)
df[bool_columns] = df[bool_columns].astype(int)
df

In [ ]:
df.columns

In [ ]:

# Select the relevant columns
df13 = df[['Yield strength (MPa)', 'Ultimate tensile strength (MPa)']]

# Compute the correlation matrix
corr2 = df13.corr()

# Plot the heatmap
sns.heatmap(corr2, 
            xticklabels=corr2.columns.values, 
            yticklabels=corr2.columns.values, 
            annot=True, cmap='coolwarm')
